In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error as mse
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import PredefinedSplit

In [2]:
# x_train = pd.read_csv('Xtrain.csv').drop(columns=['Unnamed: 0']).to_numpy()
# columns_except_evi = np.arange(x_train.shape[1]) != 4
# x_train_wo_evi = x_train[:, columns_except_evi]
# y_train = pd.read_csv('ytrain.csv').drop(columns=['Unnamed: 0']).to_numpy().ravel()

# x_val = pd.read_csv('Xval.csv').drop(columns=['Unnamed: 0']).to_numpy()
# x_val_wo_evi = x_val[:, columns_except_evi]
# y_val = pd.read_csv('yval.csv').drop(columns=['Unnamed: 0']).to_numpy().ravel()

# # For RandomSeachCV, we will need to combine training and validation 
# # sets then specify which portion is training and which is validation
# # Also, for the final performance evaluation, train on all of 
# # the training AND validation data
# X_train_plus_val = np.concatenate((x_train, x_val), axis=0)
# x_train_plus_val_wo_evi = X_train_plus_val[:, columns_except_evi]
# y_train_plus_val = np.concatenate((y_train, y_val), axis=0)

# # Create a predefined train/test split for RandomSearchCV (to be used later)
# validation_fold = np.concatenate((-1 * np.ones(len(y_train)), np.zeros(len(y_val))))
# train_val_split = PredefinedSplit(validation_fold)

# x_test = pd.read_csv('XTopTestTrain.csv').drop(columns=['Unnamed: 0']).to_numpy()
# x_test_wo_evi = x_test[:, columns_except_evi]
# y_test = pd.read_csv('yTopTestTrain.csv').drop(columns=['Unnamed: 0']).to_numpy().ravel()

In [2]:
X_train = np.loadtxt("X_train.txt")
x_train_wo_evi = X_train[:, :256]
y_train = np.loadtxt('y_train.txt')

X_val = np.loadtxt("X_val.txt")
x_val_wo_evi = X_val[:, :256]
y_val = np.loadtxt('y_val.txt')

X_test = np.loadtxt("X_test.txt")
x_test_wo_evi = X_test[:, :256]
y_test = np.loadtxt('y_test.txt')


# For RandomSeachCV, we will need to combine training and validation 
# sets then specify which portion is training and which is validation
# Also, for the final performance evaluation, train on all of 
# the training AND validation data
X_train_plus_val = np.concatenate((X_train, X_val), axis=0)
x_train_plus_val_wo_evi = X_train_plus_val[:, :256]
y_train_plus_val = np.concatenate((y_train, y_val), axis=0)
# Create a predefined train/test split for RandomSearchCV
validation_fold = np.concatenate((-1 * np.ones(len(y_train)), np.zeros(len(y_val))))
train_val_split = PredefinedSplit(validation_fold)

### I. Linear Regression Baseline

### II. BASIC RANDOM FOREST WITH ALL BANDS (TABLE 1)

In [7]:
# DEFAULT RANDOM FOREST WITH ALL BANDS except EVI
rf_model_base = RandomForestRegressor(random_state=1)

rf_model_base.fit(x_train_wo_evi, y_train)

print(f"""
Training MSE: {mse(y_train, rf_model_base.predict(x_train_wo_evi))}
Training RMSE: {np.sqrt(mse(y_train, rf_model_base.predict(x_train_wo_evi)))}
Validation MSE: {mse(y_val, rf_model_base.predict(x_val_wo_evi))}
Validation RMSE: {np.sqrt(mse(y_val, rf_model_base.predict(x_val_wo_evi)))}
Test MSE: {mse(y_test, rf_model_base.predict(x_test_wo_evi))}
Test RMSE: {np.sqrt(mse(y_test, rf_model_base.predict(x_test_wo_evi)))}
""")


Training MSE: 462.1963941051408
Training RMSE: 21.49875331513762
Validation MSE: 4192.235738553124
Validation RMSE: 64.74747669641748
Test MSE: 4527.891000213601
Test RMSE: 67.2896054395744



In [17]:
# xgboost WITH ALL BANDS except EVI
import xgboost as xgb
# XGBoost with all bands
xgb_default = xgb.XGBRegressor() #tuning learning rate did not help?

xgb_default.fit(x_train_wo_evi, y_train)

print(f"""
Training MSE: {mse(y_train, xgb_default.predict(x_train_wo_evi))}
Training RMSE: {np.sqrt(mse(y_train, xgb_default.predict(x_train_wo_evi)))}
Validation MSE: {mse(y_val, xgb_default.predict(x_val_wo_evi))}
Validation RMSE: {np.sqrt(mse(y_val, xgb_default.predict(x_val_wo_evi)))}
Test MSE: {mse(y_test, xgb_default.predict(x_test_wo_evi))}
Test RMSE: {np.sqrt(mse(y_test, xgb_default.predict(x_test_wo_evi)))}
""")


Training MSE: 1246.185298294376
Training RMSE: 35.30134980839084
Validation MSE: 4283.522352320251
Validation RMSE: 65.44862376185041
Test MSE: 4491.9414928468295
Test RMSE: 67.0219478443206



In [8]:
# DEFAULT RANDOM FOREST WITH ALL BANDS with EVI
rf_model_base = RandomForestRegressor(random_state=1)

rf_model_base.fit(X_train, y_train)

print(f"""
Training MSE: {mse(y_train, rf_model_base.predict(X_train))}
Training RMSE: {np.sqrt(mse(y_train, rf_model_base.predict(X_train)))}
Validation MSE: {mse(y_val, rf_model_base.predict(X_val))}
Validation RMSE: {np.sqrt(mse(y_val, rf_model_base.predict(X_val)))}
Test MSE: {mse(y_test, rf_model_base.predict(X_test))}
Test RMSE: {np.sqrt(mse(y_test, rf_model_base.predict(X_test)))}
""")


Training MSE: 452.75563768530196
Training RMSE: 21.27805530788239
Validation MSE: 4159.0897152675125
Validation RMSE: 64.4910049174884
Test MSE: 4547.302806571431
Test RMSE: 67.43369192452265



### III. FINE TUNING RANDOM FOREST WITH ALL BANDS (LAST TWO CODE BLOCK - TABLE 2)

In [36]:
### FINE TUNING on all normalized features
rf_model_ft_wo_evi = RandomForestRegressor(random_state=1)

max_depth_wo_evi = list(np.linspace(20, 40, 21, dtype=int))
max_depth_wo_evi.append(None)
# Define the hyperparameters to tune

hyperparameters_wo_evi = {
    "n_estimators": np.linspace(100, 110, 5, dtype=int),
    "max_features": [None],
    "max_depth": max_depth_wo_evi,
    # "max_leaf_nodes": np.linspace(2, 90, 89, dtype=int),
    # "min_samples_split": np.linspace(2, 25, 24, dtype=int),
    # "min_samples_leaf": np.linspace(1, 80, 80, dtype=int),
    # "bootstrap": [True, False], #Random Search always returned True
}

# Define the random search object
random_search_wo_evi = RandomizedSearchCV(
    estimator=rf_model_ft_wo_evi,
    param_distributions=hyperparameters_wo_evi,
    n_iter=105,
    cv=train_val_split,
    # cv=5,
    random_state=42,
    n_jobs=-1,
)

# Fit the random search object to the training data
random_search_wo_evi.fit(x_train_plus_val_wo_evi, y_train_plus_val)
# summarize results
print(f"""Best: {random_search_wo_evi.best_score_} using {random_search_wo_evi.best_params_}
""")

rf_model_tuned_wo_evi = RandomForestRegressor(**random_search_wo_evi.best_params_,random_state=1).fit(x_train_wo_evi, y_train)

print(f"""
Training MSE: {mse(y_train, rf_model_tuned_wo_evi.predict(x_train_wo_evi))}
Training RMSE: {np.sqrt(mse(y_train, rf_model_tuned_wo_evi.predict(x_train_wo_evi)))}
Validation MSE: {mse(y_val, rf_model_tuned_wo_evi.predict(x_val_wo_evi))}
Validation RMSE: {np.sqrt(mse(y_val, rf_model_tuned_wo_evi.predict(x_val_wo_evi)))}
Test MSE: {mse(y_test, rf_model_tuned_wo_evi.predict(x_test_wo_evi))}
Test RMSE: {np.sqrt(mse(y_test, rf_model_tuned_wo_evi.predict(x_test_wo_evi)))}
""")


KeyboardInterrupt: 

In [3]:
### FINE TUNING on EVI bands
rf_model_ft = RandomForestRegressor(random_state=1)


max_depth = list(np.linspace(20, 40, 21, dtype=int))
max_depth.append(None)
# Define the hyperparameters to tune

hyperparameters = {
    "n_estimators": np.linspace(95, 105, 5, dtype=int),
    "max_features": [None],
    "max_depth": max_depth,
    # "max_leaf_nodes": np.linspace(2, 50, 49, dtype=int),
    # "min_samples_split": np.linspace(2, 25, 24, dtype=int),
    # "min_samples_leaf": np.linspace(1, 30, 30, dtype=int),
    # "bootstrap": [True, False],
}

# Define the random search object
random_search = RandomizedSearchCV(
    estimator=rf_model_ft,
    param_distributions=hyperparameters,
    n_iter=105,
    cv=train_val_split,
    random_state=42,
    n_jobs=-1,
)

# Fit the random search object to the training data
random_search.fit(X_train_plus_val, y_train_plus_val)
# summarize results
print(f"""Best: {random_search.best_score_} using {random_search.best_params_}
""")
means = random_search.cv_results_['mean_test_score']
stds = random_search.cv_results_['std_test_score']
params = random_search.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))


Best: 0.0021782424149086044 using {'n_estimators': 105, 'max_features': None, 'max_depth': 20}

-0.013972 (0.000000) with: {'n_estimators': 102, 'max_features': None, 'max_depth': 35}
0.000035 (0.000000) with: {'n_estimators': 95, 'max_features': None, 'max_depth': 22}
0.002178 (0.000000) with: {'n_estimators': 105, 'max_features': None, 'max_depth': 20}
-0.014306 (0.000000) with: {'n_estimators': 105, 'max_features': None, 'max_depth': 36}
-0.009067 (0.000000) with: {'n_estimators': 105, 'max_features': None, 'max_depth': 32}
-0.010643 (0.000000) with: {'n_estimators': 102, 'max_features': None, 'max_depth': 33}
-0.004823 (0.000000) with: {'n_estimators': 95, 'max_features': None, 'max_depth': 26}
-0.008525 (0.000000) with: {'n_estimators': 95, 'max_features': None, 'max_depth': 29}
-0.017358 (0.000000) with: {'n_estimators': 97, 'max_features': None, 'max_depth': 39}
0.000162 (0.000000) with: {'n_estimators': 97, 'max_features': None, 'max_depth': 22}
-0.013618 (0.000000) with: {'n_e

In [6]:
rf_model_tuned = RandomForestRegressor(**random_search.best_params_,random_state=1).fit(X_train_plus_val, y_train_plus_val)

print(f"""
Training MSE: {mse(y_train, rf_model_tuned.predict(X_train))}
Training RMSE: {np.sqrt(mse(y_train, rf_model_tuned.predict(X_train)))}
Validation MSE: {mse(y_val, rf_model_tuned.predict(X_val))}
Validation RMSE: {np.sqrt(mse(y_val, rf_model_tuned.predict(X_val)))}
Test MSE: {mse(y_test, rf_model_tuned.predict(X_test))}
Test RMSE: {np.sqrt(mse(y_test, rf_model_tuned.predict(X_test)))}
""")


Training MSE: 1567.79639901647
Training RMSE: 39.595408812341745
Validation MSE: 1731.190936815756
Validation RMSE: 41.60758268411848
Test MSE: 4307.73812866046
Test RMSE: 65.63336139998057



### IV. SPECTURAL BANDS

In [3]:
# tuned_parameters_rf = {'n_estimators': 105, 'max_features': None, 'max_depth': 20}

# define a function to try different bands
def try_bands(x_train: np.ndarray, x_val: np.ndarray, x_test: np.ndarray):
    rf = RandomForestRegressor(
        # **tuned_parameters_rf,
        random_state=1).fit(x_train, y_train)
    print(f"""
    Training MSE: {mse(y_train, rf.predict(x_train))}
    Training RMSE: {np.sqrt(mse(y_train, rf.predict(x_train)))}
    Validation MSE: {mse(y_val, rf.predict(x_val))}
    Validation RMSE: {np.sqrt(mse(y_val, rf.predict(x_val)))}
    Test MSE: {mse(y_test, rf.predict(x_test))}
    Test RMSE: {np.sqrt(mse(y_test, rf.predict(x_test)))}
    """)

In [4]:
# RBG
try_bands(x_train_wo_evi[:,17:64], x_val_wo_evi[:,17:64], x_test_wo_evi[:,17:64])


    Training MSE: 455.5206926059722
    Training RMSE: 21.34293074078563
    Validation MSE: 4147.357015191159
    Validation RMSE: 64.39997682601414
    Test MSE: 4191.433815685806
    Test RMSE: 64.74128370433974
    


In [6]:
# No Coastal Aerosol
try_bands(x_train_wo_evi[:,17:], x_val_wo_evi[:,17:], x_test_wo_evi[:,17:])


    Training MSE: 458.64206815753886
    Training RMSE: 21.415930242638044
    Validation MSE: 4190.5807992027285
    Validation RMSE: 64.73469548242834
    Test MSE: 4867.281111348097
    Test RMSE: 69.76590221123853
    


In [32]:
x_train_wo_evi[:, np.r_[0:16, 32:256]].shape #no Blue

(25036, 240)

In [33]:
# No Blue
try_bands(x_train_wo_evi[:, np.r_[0:16, 32:256]], x_val_wo_evi[:, np.r_[0:16, 32:256]], x_test_wo_evi[:, np.r_[0:16, 32:256]])


    Training MSE: 462.2094971754447
    Training RMSE: 21.49905805321351
    Validation MSE: 4196.266437338757
    Validation RMSE: 64.77859551841763
    Test MSE: 4475.917193070232
    Test RMSE: 66.90229587293871
    


In [34]:
# No Green
try_bands(x_train_wo_evi[:, np.r_[0:32, 48:256]], x_val_wo_evi[:, np.r_[0:32, 48:256]], x_test_wo_evi[:, np.r_[0:32, 48:256]])


    Training MSE: 461.6710918815953
    Training RMSE: 21.486532802702147
    Validation MSE: 4192.817289091982
    Validation RMSE: 64.75196745344486
    Test MSE: 4510.330752176373
    Test RMSE: 67.15899606289818
    


In [35]:
# No Red
try_bands(x_train_wo_evi[:, np.r_[0:48, 64:256]], x_val_wo_evi[:, np.r_[0:48, 64:256]], x_test_wo_evi[:, np.r_[0:48, 64:256]])


    Training MSE: 459.5435994385777
    Training RMSE: 21.436968056107602
    Validation MSE: 4192.80968415727
    Validation RMSE: 64.751908729838
    Test MSE: 4536.943275440376
    Test RMSE: 67.35683540250668
    


In [36]:
# No Vegetation_Red_Edge
try_bands(x_train_wo_evi[:, np.r_[0:64, 80:256]], x_val_wo_evi[:, np.r_[0:64, 80:256]], x_test_wo_evi[:, np.r_[0:64, 80:256]])


    Training MSE: 457.2574993544991
    Training RMSE: 21.383580134170682
    Validation MSE: 4170.841000838432
    Validation RMSE: 64.58204859586317
    Test MSE: 4548.188138096123
    Test RMSE: 67.44025606487659
    


In [37]:
# No Vegetation_Red_Edge 2
try_bands(x_train_wo_evi[:, np.r_[0:80, 96:256]], x_val_wo_evi[:, np.r_[0:80, 96:256]], x_test_wo_evi[:, np.r_[0:80, 96:256]])


    Training MSE: 459.6758325780287
    Training RMSE: 21.440052065655735
    Validation MSE: 4184.2021200265335
    Validation RMSE: 64.68540886495605
    Test MSE: 4496.730550443227
    Test RMSE: 67.0576658588951
    


In [38]:
# Vegetation_Red_Edge 3
try_bands(x_train_wo_evi[:, np.r_[0:96, 112:256]], x_val_wo_evi[:, np.r_[0:96, 112:256]], x_test_wo_evi[:, np.r_[0:96, 112:256]])


    Training MSE: 460.2877855716123
    Training RMSE: 21.454318576259006
    Validation MSE: 4183.494782611296
    Validation RMSE: 64.67994111477913
    Test MSE: 4523.706489340344
    Test RMSE: 67.25850495915252
    


In [39]:
# No NIR
try_bands(x_train_wo_evi[:, np.r_[0:112, 128:256]], x_val_wo_evi[:, np.r_[0:112, 128:256]], x_test_wo_evi[:, np.r_[0:112, 128:256]])


    Training MSE: 461.0005189778878
    Training RMSE: 21.470922639185485
    Validation MSE: 4198.862049209701
    Validation RMSE: 64.79862690836667
    Test MSE: 4596.254750726961
    Test RMSE: 67.79568386502906
    


In [40]:
# No Narrow NIR
try_bands(x_train_wo_evi[:, np.r_[0:128, 144:256]], x_val_wo_evi[:, np.r_[0:128, 144:256]], x_test_wo_evi[:, np.r_[0:128, 144:256]])


    Training MSE: 460.4022349759341
    Training RMSE: 21.4569856917493
    Validation MSE: 4164.223776543063
    Validation RMSE: 64.53079711690428
    Test MSE: 4537.31820718358
    Test RMSE: 67.35961852017557
    


In [41]:
# No water vapor
try_bands(x_train_wo_evi[:, np.r_[0:144, 160:256]], x_val_wo_evi[:, np.r_[0:144, 160:256]], x_test_wo_evi[:, np.r_[0:144, 160:256]])


    Training MSE: 459.5198333833573
    Training RMSE: 21.436413724859793
    Validation MSE: 4181.307688458906
    Validation RMSE: 64.66303185328466
    Test MSE: 4522.040616116803
    Test RMSE: 67.24611971048444
    


In [42]:
# no SWIR1
try_bands(x_train_wo_evi[:, np.r_[0:160, 176:256]], x_val_wo_evi[:, np.r_[0:160, 176:256]], x_test_wo_evi[:, np.r_[0:160, 176:256]])


    Training MSE: 459.5306456254078
    Training RMSE: 21.436665916727996
    Validation MSE: 4170.312150577251
    Validation RMSE: 64.57795406001378
    Test MSE: 4470.855320594862
    Test RMSE: 66.86445483659358
    


In [43]:
# no SWIR2
try_bands(x_train_wo_evi[:, np.r_[0:176, 192:256]], x_val_wo_evi[:, np.r_[0:176, 192:256]], x_test_wo_evi[:, np.r_[0:176, 192:256]])


    Training MSE: 460.4577072711947
    Training RMSE: 21.45827829233265
    Validation MSE: 4185.676323575913
    Validation RMSE: 64.69680303984049
    Test MSE: 4533.730668532853
    Test RMSE: 67.33298351129892
    


### IV. Environmental factors


In [45]:
# no Cloud
try_bands(x_train_wo_evi[:, np.r_[0:192, 208:256]], x_val_wo_evi[:, np.r_[0:192, 208:256]], x_test_wo_evi[:, np.r_[0:192, 208:256]])


    Training MSE: 460.9885446581468
    Training RMSE: 21.470643787696417
    Validation MSE: 4208.196193839221
    Validation RMSE: 64.87061117208025
    Test MSE: 4312.119656026667
    Test RMSE: 65.66673172944324
    


In [46]:
# no Latitude
try_bands(x_train_wo_evi[:, np.r_[0:224, 240:256]], x_val_wo_evi[:, np.r_[0:224, 240:256]], x_test_wo_evi[:, np.r_[0:224, 240:256]])


    Training MSE: 460.19613958208953
    Training RMSE: 21.45218262979526
    Validation MSE: 4170.42388352118
    Validation RMSE: 64.57881915551863
    Test MSE: 4531.637392623065
    Test RMSE: 67.31743750784833
    


In [51]:
# no Longitude
try_bands(x_train_wo_evi[:, :240], x_val_wo_evi[:, :240], x_test_wo_evi[:, :240])


    Training MSE: 460.18034125641486
    Training RMSE: 21.451814404763407
    Validation MSE: 4178.267520486809
    Validation RMSE: 64.6395198039621
    Test MSE: 4537.764995701152
    Test RMSE: 67.36293488040104
    
